### Cifar-10 이미지 분류 - v2
- Canadian Institute for Advanced Research 10
- 데이터 부풀리기 사용

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
seed = 2023
np.random.seed(seed)
tf.random.set_seed(seed)

- Data preprocessing

In [2]:
from tensorflow.keras.datasets import cifar10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

170498071/170498071 [==============================] - 6s 0us/step


In [3]:
# X data scaling
X_train = X_train / 255.
X_test = X_test / 255.

In [4]:
# y data one-hot-encoding
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)

- 모델 정의/설정/학습/평가

In [5]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
model = Sequential([
    Conv2D(32, (3,3), input_shape=X_train.shape[1:], padding='same', activation='relu'),
    Conv2D(32, (3,3), activation='relu'),
    MaxPooling2D(),
    Dropout(0.25),
    Conv2D(64, (3,3), padding='same', activation='relu'),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(),
    Dropout(0.25),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(10, activation='softmax')
])

In [7]:
model.compile('adam', 'categorical_crossentropy', ['accuracy'])

model_path = 'best-cifar10-v2.h5'
mc = ModelCheckpoint(model_path, save_best_only=True, verbose=1)
es = EarlyStopping(patience=5)

# - 데이터 부풀리기

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_valid, Y_train, Y_valid = train_test_split(
    X_train, Y_train, stratify=Y_train, test_size=0.2, random_state=seed
)

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rotation_range=10,          # randomly rotata images in the range (degree 0 to 15)
    width_shift_range=0.1,      # randomly shift images horizontally
    height_shift_range=0.1,     # randomly shift images vertically
    shear_range=0.1,            # 도형의 기울기
    zoom_range=[0.8,1.2],        # 0.8~1.5 배 크기로 임의로 확대/축소
    horizontal_flip=True,       # randomly flip images
    vertical_flip=False,
    fill_mode='nearest',        # set mode for filling points outside the input boundaries
)

In [10]:
train_datagen.fit(X_train)

- 데이터 부풀리기를 한 결과로 학습

In [11]:
hist = model.fit(
    train_datagen.flow(X_train, Y_train, batch_size=32),
    validation_data=(X_valid, Y_valid),
    epochs=100, verbose=0,
    callbacks=[mc, es]
)


Epoch 1: val_loss improved from inf to 1.34754, saving model to best-cifar10-v2.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(



Epoch 2: val_loss improved from 1.34754 to 1.12170, saving model to best-cifar10-v2.h5

Epoch 3: val_loss improved from 1.12170 to 0.98904, saving model to best-cifar10-v2.h5

Epoch 4: val_loss improved from 0.98904 to 0.98046, saving model to best-cifar10-v2.h5

Epoch 5: val_loss did not improve from 0.98046

Epoch 6: val_loss improved from 0.98046 to 0.90545, saving model to best-cifar10-v2.h5

Epoch 7: val_loss improved from 0.90545 to 0.86690, saving model to best-cifar10-v2.h5

Epoch 8: val_loss did not improve from 0.86690

Epoch 9: val_loss did not improve from 0.86690

Epoch 10: val_loss improved from 0.86690 to 0.79748, saving model to best-cifar10-v2.h5

Epoch 11: val_loss improved from 0.79748 to 0.78376, saving model to best-cifar10-v2.h5

Epoch 12: val_loss improved from 0.78376 to 0.76763, saving model to best-cifar10-v2.h5

Epoch 13: val_loss improved from 0.76763 to 0.73432, saving model to best-cifar10-v2.h5

Epoch 14: val_loss did not improve from 0.73432

Epoch 15: 

In [12]:
best_model = load_model(model_path)
best_model.evaluate(X_test, Y_test)

313/313 [==============================] - 1s 4ms/step - loss: 0.6659 - accuracy: 0.7705


[0.6659010052680969, 0.7705000042915344]